In [0]:
%sql
DROP TABLE IF EXISTS demo;
CREATE OR REPLACE TABLE demo(id INT,text STRING);

INSERT INTO demo VALUES (1,"Back");
INSERT INTO demo VALUES (2,"to the");
INSERT INTO demo VALUES (3,"future");

SELECT * FROM demo;

id,text
2,to the
3,future
1,Back


lets mess up with the data

In [0]:
%sql
DELETE FROM demo;

SELECT * FROM demo;

id,text


In [0]:
%sql
DESCRIBE HISTORY demo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2025-02-03T07:50:42Z,5755764547042441,inaya998877@gmail.com,DELETE,"Map(predicate -> [""true""])",null,List(2497773069427497),0203-052121-nggyovx3,3,WriteSerializable,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 2136, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 48, numDeletionVectorsUpdated -> 0, numDeletedRows -> 3, scanTimeMs -> 40, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/15.4.x-scala2.12
3,2025-02-03T07:50:40Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
2,2025-02-03T07:50:37Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
1,2025-02-03T07:50:36Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 702)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-02-03T07:50:33Z,5755764547042441,inaya998877@gmail.com,CREATE OR REPLACE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {}, statsOnLoad -> false)",null,List(2497773069427497),0203-052121-nggyovx3,null,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12


#View Previous data

In [0]:
%sql
SELECT * FROM demo@v3
-- in v4 data was deleted which means tille v3 we have the data as we can see

id,text
2,to the
3,future
1,Back


In [0]:
%python
spark.read.option("versionAsOf",3)\
          .table("demo")\
          .display()

# same we can run in python

id,text
2,to the
3,future
1,Back


In [0]:
%sql
-- SELECT * FROM demo TIMESTAMP AS OF '2025-01-29T11:58:19.000+00:00'

--  we can query the data using timestamp of that version also

#Restore table

In [0]:
%sql
RESTORE TABLE demo TO VERSION AS OF 3;
-- RESTORE TABLE demo TIMESTAMP AS OF "2025-01-29T11:58:19.000+00:00";

SELECT * FROM demo;

id,text
2,to the
3,future
1,Back


In [0]:
%sql
DESCRIBE HISTORY demo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2025-02-03T07:50:56Z,5755764547042441,inaya998877@gmail.com,RESTORE,"Map(version -> 3, timestamp -> null)",null,List(2497773069427497),0203-052121-nggyovx3,4,Serializable,false,"Map(numRestoredFiles -> 3, removedFilesSize -> 0, numRemovedFiles -> 0, restoredFilesSize -> 2136, numOfFilesAfterRestore -> 3, tableSizeAfterRestore -> 2136)",null,Databricks-Runtime/15.4.x-scala2.12
4,2025-02-03T07:50:42Z,5755764547042441,inaya998877@gmail.com,DELETE,"Map(predicate -> [""true""])",null,List(2497773069427497),0203-052121-nggyovx3,3,WriteSerializable,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 2136, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 48, numDeletionVectorsUpdated -> 0, numDeletedRows -> 3, scanTimeMs -> 40, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/15.4.x-scala2.12
3,2025-02-03T07:50:40Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
2,2025-02-03T07:50:37Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
1,2025-02-03T07:50:36Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 702)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-02-03T07:50:33Z,5755764547042441,inaya998877@gmail.com,CREATE OR REPLACE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {}, statsOnLoad -> false)",null,List(2497773069427497),0203-052121-nggyovx3,null,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12


# Time travel and vaccum
- So here to time travel we needed delta_log and parquet files to traceback the data
- delta log and unnecessary parquet filed can become very heavy
- So we have additional utility which is very very dangerous but required indeed that is #**Vaccum**
- **Vaccum** deleted unnecessary files from the storage older than 7 days
- Files deleted through vaccum are permanently deleted files so its advicable to of dry run before

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled=True

key,value
spark.databricks.delta.retentionDurationCheck.enabled,True


In [0]:
%sql
DESC HISTORY demo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2025-02-03T07:50:56Z,5755764547042441,inaya998877@gmail.com,RESTORE,"Map(version -> 3, timestamp -> null)",null,List(2497773069427497),0203-052121-nggyovx3,4,Serializable,false,"Map(numRestoredFiles -> 3, removedFilesSize -> 0, numRemovedFiles -> 0, restoredFilesSize -> 2136, numOfFilesAfterRestore -> 3, tableSizeAfterRestore -> 2136)",null,Databricks-Runtime/15.4.x-scala2.12
4,2025-02-03T07:50:42Z,5755764547042441,inaya998877@gmail.com,DELETE,"Map(predicate -> [""true""])",null,List(2497773069427497),0203-052121-nggyovx3,3,WriteSerializable,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 2136, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 48, numDeletionVectorsUpdated -> 0, numDeletedRows -> 3, scanTimeMs -> 40, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/15.4.x-scala2.12
3,2025-02-03T07:50:40Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
2,2025-02-03T07:50:37Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
1,2025-02-03T07:50:36Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 702)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-02-03T07:50:33Z,5755764547042441,inaya998877@gmail.com,CREATE OR REPLACE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {}, statsOnLoad -> false)",null,List(2497773069427497),0203-052121-nggyovx3,null,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
%sql
VACUUM demo RETAIN 0 HOURS DRY RUN
--  Its by default not allowed to vacuum the data just created or 0 hours before atleast >168 hours is req.
-- if we remove retain 0 hourse it wont show any file cause it shows file >7 days

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: java.lang.IllegalArgumentException: requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.
If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false
If you are not sure, please use a value not less than "168 hours".
       
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.checkRetentionPeriodSafety(VacuumCommand.scala:113)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.getValidFilesFromSnapshot(VacuumCommand.scala:133)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.$anonfun$gc$2(V

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled=False

key,value
spark.databricks.delta.retentionDurationCheck.enabled,False


In [0]:
%sql
VACUUM demo RETAIN 0 HOURS DRY RUN

path


In [0]:
%sql
INSERT INTO demo VALUES (9,"vACUUM");

DELETE FROM demo WHERE id=9;

num_affected_rows
1


In [0]:
%sql
desc history demo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
7,2025-02-03T07:51:43Z,5755764547042441,inaya998877@gmail.com,DELETE,"Map(predicate -> [""(id#193343 = 9)""])",null,List(2497773069427497),0203-052121-nggyovx3,6,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 717, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1734, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 1243, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 491)",null,Databricks-Runtime/15.4.x-scala2.12
6,2025-02-03T07:51:39Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,5,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
5,2025-02-03T07:50:56Z,5755764547042441,inaya998877@gmail.com,RESTORE,"Map(version -> 3, timestamp -> null)",null,List(2497773069427497),0203-052121-nggyovx3,4,Serializable,false,"Map(numRestoredFiles -> 3, removedFilesSize -> 0, numRemovedFiles -> 0, restoredFilesSize -> 2136, numOfFilesAfterRestore -> 3, tableSizeAfterRestore -> 2136)",null,Databricks-Runtime/15.4.x-scala2.12
4,2025-02-03T07:50:42Z,5755764547042441,inaya998877@gmail.com,DELETE,"Map(predicate -> [""true""])",null,List(2497773069427497),0203-052121-nggyovx3,3,WriteSerializable,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 2136, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 48, numDeletionVectorsUpdated -> 0, numDeletedRows -> 3, scanTimeMs -> 40, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/15.4.x-scala2.12
3,2025-02-03T07:50:40Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
2,2025-02-03T07:50:37Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
1,2025-02-03T07:50:36Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 702)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-02-03T07:50:33Z,5755764547042441,inaya998877@gmail.com,CREATE OR REPLACE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {}, statsOnLoad -> false)",null,List(2497773069427497),0203-052121-nggyovx3,null,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
%fs ls dbfs:/user/hive/warehouse/demo/

path,name,size,modificationTime
dbfs:/user/hive/warehouse/demo/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/demo/part-00000-5e2e07df-1a60-4790-9454-48d611b58fe6-c000.snappy.parquet,part-00000-5e2e07df-1a60-4790-9454-48d611b58fe6-c000.snappy.parquet,702,1738569036000
dbfs:/user/hive/warehouse/demo/part-00000-738180ad-d566-4e0b-ab97-a5112b64e440-c000.snappy.parquet,part-00000-738180ad-d566-4e0b-ab97-a5112b64e440-c000.snappy.parquet,717,1738569037000
dbfs:/user/hive/warehouse/demo/part-00000-899790aa-e48b-4c26-943c-31e3e9e51e69-c000.snappy.parquet,part-00000-899790aa-e48b-4c26-943c-31e3e9e51e69-c000.snappy.parquet,717,1738569099000
dbfs:/user/hive/warehouse/demo/part-00000-e0e8c5ef-4969-4c27-bbeb-50c3334066ca-c000.snappy.parquet,part-00000-e0e8c5ef-4969-4c27-bbeb-50c3334066ca-c000.snappy.parquet,717,1738569039000


In [0]:
%sql
VACUUM demo RETAIN 0 HOURS DRY RUN

path
dbfs:/user/hive/warehouse/demo/part-00000-899790aa-e48b-4c26-943c-31e3e9e51e69-c000.snappy.parquet


In [0]:
%sql
VACUUM demo RETAIN 0 HOURS

path
dbfs:/user/hive/warehouse/demo


In [0]:
%fs ls dbfs:/user/hive/warehouse/demo/

path,name,size,modificationTime
dbfs:/user/hive/warehouse/demo/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/demo/part-00000-5e2e07df-1a60-4790-9454-48d611b58fe6-c000.snappy.parquet,part-00000-5e2e07df-1a60-4790-9454-48d611b58fe6-c000.snappy.parquet,702,1738569036000
dbfs:/user/hive/warehouse/demo/part-00000-738180ad-d566-4e0b-ab97-a5112b64e440-c000.snappy.parquet,part-00000-738180ad-d566-4e0b-ab97-a5112b64e440-c000.snappy.parquet,717,1738569037000
dbfs:/user/hive/warehouse/demo/part-00000-e0e8c5ef-4969-4c27-bbeb-50c3334066ca-c000.snappy.parquet,part-00000-e0e8c5ef-4969-4c27-bbeb-50c3334066ca-c000.snappy.parquet,717,1738569039000


In [0]:
%sql
DESCRIBE HISTORY demo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
7,2025-02-03T07:51:43Z,5755764547042441,inaya998877@gmail.com,DELETE,"Map(predicate -> [""(id#193343 = 9)""])",null,List(2497773069427497),0203-052121-nggyovx3,6,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 717, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1734, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 1243, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 491)",null,Databricks-Runtime/15.4.x-scala2.12
6,2025-02-03T07:51:39Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,5,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
5,2025-02-03T07:50:56Z,5755764547042441,inaya998877@gmail.com,RESTORE,"Map(version -> 3, timestamp -> null)",null,List(2497773069427497),0203-052121-nggyovx3,4,Serializable,false,"Map(numRestoredFiles -> 3, removedFilesSize -> 0, numRemovedFiles -> 0, restoredFilesSize -> 2136, numOfFilesAfterRestore -> 3, tableSizeAfterRestore -> 2136)",null,Databricks-Runtime/15.4.x-scala2.12
4,2025-02-03T07:50:42Z,5755764547042441,inaya998877@gmail.com,DELETE,"Map(predicate -> [""true""])",null,List(2497773069427497),0203-052121-nggyovx3,3,WriteSerializable,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 2136, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 48, numDeletionVectorsUpdated -> 0, numDeletedRows -> 3, scanTimeMs -> 40, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/15.4.x-scala2.12
3,2025-02-03T07:50:40Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
2,2025-02-03T07:50:37Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
1,2025-02-03T07:50:36Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2497773069427497),0203-052121-nggyovx3,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 702)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-02-03T07:50:33Z,5755764547042441,inaya998877@gmail.com,CREATE OR REPLACE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {}, statsOnLoad -> false)",null,List(2497773069427497),0203-052121-nggyovx3,null,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
%sql
SELECT * FROM demo@v6

org.apache.spark.SparkException: [FAILED_READ_FILE.DBR_FILE_NOT_EXIST] Error while reading file dbfs:/user/hive/warehouse/demo/part-00000-899790aa-e48b-4c26-943c-31e3e9e51e69-c000.snappy.parquet. [DELTA_FILE_NOT_FOUND_DETAILED] File dbfs:/user/hive/warehouse/demo/part-00000-899790aa-e48b-4c26-943c-31e3e9e51e69-c000.snappy.parquet referenced in the transaction log cannot be found. This occurs when data has been manually deleted from the file system rather than using the table `DELETE` statement. For more information, see https://docs.databricks.com/delta/delta-intro.html#frequently-asked-questions SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.fileNotExistErrorDBR(QueryExecutionErrors.scala:1069)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logMissingFileNameAndThrow(FileScanRDD.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:676)
	at org.apache.spark.util.NextIterator.ha

In [0]:
%sql
ALTER TABLE demo
  SET TBLPROPERTIES ('delta.deletedFileRetentionDuration'='interval 14 days')

In [0]:
%sql
desc extended demo

col_name,data_type,comment
id,int,null
text,string,null
,,
# Delta Statistics Columns,,
Column Names,"id, text",
Column Selection Method,first-32,
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,default,
